Stap 1: generaliseer, zoals al gedaan voor zittingsdagen, 2. voeg text slot indirection toe 3. test met toevoegen van named entities (en dus nieuwe text slots) 4. isoleer/modelleer die indirectie netjes.

Stap 1: generalisatie doorvoeren. Te implementeren functies: get_file_sequence_for_container(text_container), get_root_tree_element(file), traverse(node,node_label,text,annotations), (offset_text_coordinates is in dit geval triviaal, want maar 1 file) - DONE

Stap 2: voeg een class toe, die anchors en fragmenten encapsuleert - done

Stap 3: pas parsing code aan: gebruik SegmentedText in plaats van een gewone list - done

Daarna: test dit met het toevoegen van named entities, die paragraafteksten opsplitsen - DONE

Volgende stappen
- pas API functies aan om met anchors om te gaan in plaats van met indexen.
- check of dit ook netjes werkt als een entity door een page break wordt gesplitst
- DENK NA: hoe bruikbaar is dit nog als er veel achteraf wordt geannoteerd? Of is het daarvoor beter al een fijnmazige basissegmentatie klaar te zetten? Beide strategien hebben misschien bestaansrecht, wanneer is welke strategie het beste?

In [1]:
l = ["nul", "een", "twee", "drie","vier", "vijf"]
anchors = ['n0', 'n1','n2','n3','n4','n5']

index_at_after_anchor = anchors.index('n0')
print(index_at_after_anchor)

text_to_split = l[index_at_after_anchor]
print(text_to_split)

at_char_offset = 2
t1 = text_to_split[:at_char_offset]
t2 = text_to_split[at_char_offset:]
print(f"t1: {t1}, t2: {t2}")

l[index_at_after_anchor] = t1
l.insert(index_at_after_anchor + 1, t2)

anchors.insert(index_at_after_anchor + 1, 'n6')
print(l)
print(anchors)


0
nul
t1: nu, t2: l
['nu', 'l', 'een', 'twee', 'drie', 'vier', 'vijf']
['n0', 'n6', 'n1', 'n2', 'n3', 'n4', 'n5']


In [2]:
# For large lists, consider implementation on basis of blist (pip install blist). This is supposed
# to have performance O(log n) instead of O(n) for insert operations.

import uuid

class SegmentedText:    
    def __init__(self):
        self._ordered_segments = []
        self._anchors = []
        
    def append(self, text_element):        
     #   self._anchors[self._new_anchor_id()] = len(self._ordered_segments)
        self._anchors.append(self._new_anchor_id())
        self._ordered_segments.append(text_element)
        return
        
    def extend(self, textelement_list):
        if isinstance(textelement_list, list):
            # add a number of new text segment to self
            for te in textelement_list:
                self.append(te)
        else:
            # textelement_list is a SegmentedText object
            self._ordered_segments.extend(textelement_list._ordered_segments)
            self._anchors.extend(textelement_list._anchors)
        return
    
    def len(self):
        return len(self._ordered_segments)
        
    def element_at(self, anchor):
        # return self._ordered_segments[self._anchors[anchor]]
        return self._ordered_segments[self._anchors.index(anchor)]
    
    # so far, only one variation of slicing is supported, add other flavours as well (search for sample code)
    # remark: may go wrong at end of lists, not tested yet
    def slice(self, from_anchor, to_anchor):
        from_index = self._anchors.index(from_anchor)
        to_index = self._anchors.index(to_anchor)
        return self._ordered_segments[from_index:to_index+1]
    
    # remark: split a list element is potentially an expensive operation: the complete list might be recreated.
    def split(self, after_anchor, at_char_offset):
        index_at_after_anchor = self._anchors.index(after_anchor)
        text_to_split = self._ordered_segments[index_at_after_anchor]

        t1 = text_to_split[:at_char_offset]
        t2 = text_to_split[at_char_offset:]

        self._ordered_segments[index_at_after_anchor] = t1
        self._ordered_segments.insert(index_at_after_anchor + 1, t2)

        new_anchor_id = self._new_anchor_id()
        self._anchors.insert(index_at_after_anchor + 1, new_anchor_id)
        
        return new_anchor_id
    
    def _new_anchor_id(self):
        return 'anchor_' + str(uuid.uuid4())
        
    def __repr__(self):
        return str(self._anchors)
        
    def __str__(self):
        return str(self._ordered_segments)
    
sg = SegmentedText()
sg.append("mijn eerste segment")
sg.append("mijn tweede segment")
sg.extend(['derde', 'vierde'])
print(sg)
sg



['mijn eerste segment', 'mijn tweede segment', 'derde', 'vierde']


['anchor_3d3ad5e0-402b-486f-b425-fd970cdd2b81', 'anchor_91e0b525-5d2c-4ca2-b000-546a63728f41', 'anchor_0ea2d7d0-5ba6-42cb-81be-7bc27e4ef1dd', 'anchor_ca136d2f-ca58-4096-9743-8fce95954ce6']

In [3]:
sg.split('anchor_ff70d15f-5b47-4074-864e-686e42add4e0', 12)
print(sg)
sg

ValueError: 'anchor_ff70d15f-5b47-4074-864e-686e42add4e0' is not in list

In [4]:
import lxml
from lxml import etree
import uuid

path = '../data/tei-samples/bosb002graa04_01.xml'

_last_page_begin_index = 0
_last_section_begin_index = -1
_last_chapter_begin_index = -1
_last_paragraph_begin_index = -1
_last_head_begin_index = -1

_last_page_end_index = -1
_last_section_end_index = -1
_last_chapter_end_index = -1
_last_paragraph_end_index = -1
_last_head_end_index = -1

_last_page_id = ""

all_textelements=SegmentedText()
all_annotations=[]

def get_file_sequence_for_container(text_container):
    return [path]

def get_root_tree_element(file):
    # use iterparse to traverse the xml hierarchy, depth first, post order
    return etree.iterparse(path, events=('start','end'))

# handle each of the elements in the hierarchy according to 'layer type'
def handle_element(action,e,text,annotations):  
    global _last_page_begin_index
    global _last_section_begin_index
    global _last_chapter_begin_index
    global _last_paragraph_begin_index
    global _last_head_begin_index

    global _last_page_end_index
    global _last_section_end_index
    global _last_chapter_end_index
    global _last_paragraph_end_index
    global _last_head_end_index
    
    global _last_page_id

    if action == 'start':
        # store last begin indexes
        if e.tag == 'p':
            _last_paragraph_begin_index = text.len()               
        elif e.tag == 'div' and e.get('type') == 'chapter':
            _last_chapter_begin_index = text.len()
        elif e.tag == 'div' and e.get('type') == 'section':
            _last_section_begin_index = text.len()
        elif e.tag == 'head':
            _last_head_begin_index = text.len()
    elif action == 'end':
        if e.tag == 'p': 
            # leaf text element, add to all_textelements, also include text after possible pb's
            for index, t in enumerate(e.itertext()):
                text.append(t.strip())
                if index > 0: # assume: caused by pb contained within p. Update page end.
                    _last_page_end_index = text.len()-1
            
            _last_paragraph_end_index = text.len()-1

            if _last_paragraph_begin_index <= _last_paragraph_end_index:
                annotations.append({'label':'paragraph','begin_anchor': text._anchors[_last_paragraph_begin_index],\
                            'end_anchor':text._anchors[_last_paragraph_end_index],'id': 'annot_'+str(uuid.uuid4())})
        elif e.tag == 'head':
            # leaf text element, add to all_textelements
            text.append(e.text)
            
            _last_head_end_index = text.len()-1
            annotations.append({'label':'head','begin_anchor': text._anchors[_last_head_begin_index],\
                            'end_anchor':text._anchors[_last_head_end_index],'id': 'annot_'+str(uuid.uuid4())}) 
        elif e.tag == 'div' and e.get('type') == 'chapter':
            _last_chapter_end_index = text.len()-1
            annotations.append({'label':'chapter','begin_anchor': text._anchors[_last_chapter_begin_index],\
                            'end_anchor':text._anchors[_last_chapter_end_index],'id': 'annot_'+str(uuid.uuid4())})            
        elif e.tag == 'div' and e.get('type') == 'section':
            _last_section_end_index = text.len()-1
            annotations.append({'label':'section','begin_anchor': text._anchors[_last_section_begin_index],\
                            'end_anchor':text._anchors[_last_section_end_index],'id': 'annot_'+str(uuid.uuid4())})               
        elif e.tag == 'pb':
            # first store the 'previous' page, then store begin and end of currently closed page
            annotations.append({'label':'page','begin_anchor': text._anchors[_last_page_begin_index],\
                            'end_anchor':text._anchors[_last_page_end_index],'id': _last_page_id}) 
            _last_page_begin_index = _last_page_end_index
            _last_page_end_index = text.len()-1 
            _last_page_id = f"page-{e.get('n')}"
            
    return        

def traverse(node,node_label,text,annotations):
    for action, elem in node:
        handle_element(action,elem,text,annotations)  
    return
    
# Process per file, properly concatenate results, maintaining proper referencing the baseline text elements
for f_name in get_file_sequence_for_container('just one tei document'):
    text_segments = SegmentedText()
    annotation_array = []
            
    source_data = get_root_tree_element(f_name)

    traverse(source_data,'',text_segments,annotation_array)
           
    # properly concatenate annotation info taking ongoing line indexes into account - trivial, do not apply in this case
#    for ai in annotation_array:
#        ai['begin_index'] += len(all_textlines)
#        ai['end_index'] += len(all_textlines)
    
    all_textelements.extend(text_segments)       
    all_annotations.extend(annotation_array)
    

In [5]:
len(all_textelements._anchors)

3130

In [6]:
# return generator for annotations with a specific label
def get_annotations_of_type(type,annotations):
    return (a for a in annotations if a['label'] == type)

In [7]:
[paras for paras in get_annotations_of_type('paragraph', all_annotations)][102]

{'label': 'paragraph',
 'begin_anchor': 'anchor_f041667c-ac6f-49b8-9335-da7c15d94786',
 'end_anchor': 'anchor_f041667c-ac6f-49b8-9335-da7c15d94786',
 'id': 'annot_85f56def-8c43-4775-b989-9dddef5cfcab'}

In [8]:
sample_para = [paras for paras in get_annotations_of_type('paragraph', all_annotations)][103]
print(sample_para['begin_anchor'])
print(sample_para['end_anchor'])

anchor_85fcdcd4-6c0e-43f0-b4bb-9a74dfd86616
anchor_85fcdcd4-6c0e-43f0-b4bb-9a74dfd86616


In [9]:
all_textelements.slice(sample_para['begin_anchor'], sample_para['end_anchor'])

['‘Die goede Geertrui!’ zeide Elisabeth glimlachend tot Courtenay; ‘zij is zoo ingenomen met den titel, dien mijn vader haar gaf, toen zij mijne min werd, dat zij zelfs hare gehechtheid aan mij onder de stijve windsels der étiquette verbergt. Maar ik schaam mij, ik ben wel egoïstisch, beste Graaf! ik heb er tot hiertoe slechts aan gedacht u mijne eigene grieven te verhalen, en was daarna zoo verdiept in het luisteren naar u, dat ik er bij vergat, hoe vermoeid gij moet zijn en hoe verlangend naar eenige verkwikking.’']

In [10]:
begin_anchor = all_textelements.split(sample_para['begin_anchor'], 11)
end_anchor = all_textelements.split(begin_anchor, 8)

In [11]:
all_textelements.slice(begin_anchor, begin_anchor)

['Geertrui']

In [12]:
all_annotations.append({'label':'entity','begin_anchor': begin_anchor,'end_anchor': begin_anchor,\
                    'id': 'annot_'+str(uuid.uuid4()), 'entity_type': 'per', 'entity_text': 'Geertrui'})

In [13]:
entities = [ents for ents in get_annotations_of_type('entity', all_annotations)]
entities

[{'label': 'entity',
  'begin_anchor': 'anchor_757fc665-3264-480c-bd5a-e040d2e527f9',
  'end_anchor': 'anchor_757fc665-3264-480c-bd5a-e040d2e527f9',
  'id': 'annot_0d544ed0-74b3-4668-ab09-10c8f441f1fa',
  'entity_type': 'per',
  'entity_text': 'Geertrui'}]

In [14]:
all_textelements.slice(entities[0]['begin_anchor'], entities[0]['end_anchor'])

['Geertrui']

In [15]:
def get_annotations_overlapping_with(begin,end,annotations):
    return (a for a in annotations if (a['begin_index'] >= begin and a['begin_index'] < end) or\
           (a['end_index'] > begin and a['end_index'] <= end) or\
           (a['begin_index'] <= begin and a['end_index'] >= end))

In [16]:
# print all annotations for a chapter
for a in get_annotations_overlapping_with(636,832,all_annotations):
    print(a)

KeyError: 'begin_index'

In [10]:
# nested generators
def get_annotations_of_type_overlapping(type,begin,end,annotations):
    return get_annotations_of_type(type,(get_annotations_overlapping_with(begin,end,annotations)))

Variations of the above 3 functions, now for multiple types iso for just one type.

In [17]:
def get_annotations_of_types(types,annotations):
    return (a for a in annotations if a['label'] in types)

In [18]:
def get_annotations_of_types_overlapping(types,begin,end,annotations):
    return get_annotations_of_types(types,(get_annotations_overlapping_with(begin,end,annotations)))

In [19]:
def get_textlines_between(begin,end,annotations): 
    textlines = []
    types = ('paragraph', 'head')
    for line_annot in get_annotations_of_types_overlapping(types,begin,end,annotations):
        textlines.append((line_annot['begin_index'],line_annot['end_index'], \
                          all_textelements[line_annot['begin_index']:line_annot['end_index']]))
    return textlines

In [20]:
# get text for (text segment of) a chapter
for a in get_textlines_between(636,832,all_annotations):
    print(a)

KeyError: 'begin_index'

Gebruik bovenstaande voor het visualiseren van alle tekst voor een pagina, of alle heads (vanaf het midden van het boek)

In [21]:
for a in get_annotations_of_type('page', all_annotations):
    if a['id'] == 'page-116':
        print(a)

{'label': 'page', 'begin_anchor': 'anchor_45ba00fe-aa4b-4bbb-82ac-31cde98ebfd1', 'end_anchor': 'anchor_3b290ff8-908d-4cd8-a11b-17dbe144450b', 'id': 'page-116'}


In [16]:
for tl in get_textlines_between(1036,1042,all_annotations):
    print(tl)

(1037, 1037, [])
(1038, 1038, [])
(1039, 1039, [])
(1040, 1040, [])
(1041, 1042, ['Nog slechts ten deele hersteld van de verschillende aandoeningen, die zijn gemoed geschokt hadden, liep de Graaf van Devonshire eene der gewone hofzalen binnen. Men zegt hem dat de Koningin naar hem gevraagd heeft; men wijst hem het vertrek aan waar zij zich bevindt; hij begeeft zich daarheen; hij ziet er den Kanselier en Benefield bij haar. Eensklaps is het hem duidelijk, dat dit tot eene vreeselijke ontknooping leiden kan, dat beiden met elkander in betrekking kunnen staan en het eens tot zijn verderf. Als wij een belangrijk geheim met ons omgedragen, is het ons alsof elk onzer bewegingen dat'])


ISSUE: bovenstaande haalt alle overlappende paragrafen en heads op, maar zou moeten afbreken bij de page breaks die binnen paragrafen optreden. De kleinste basissegmenten zouden moeten worden opgehaald. Wat ontbreekt, is het equivalent van de lines, zoals in de republic casus.

Wat wel moet gebeuren: voor een end of paragraph moeten niet alle textonderdelen worden gejoined, maar allemaal afzonderlijk worden toegevoegd aan all_textelements. Vervolgens moet een paragraph annotation worden aangemaakt over al deze onderdelen.

In [17]:
for t in all_textelements[1036:1042]:
    print(f"{t}\n")

dedigen. Het ware hem wellust geweest, slechts eens den zoeten naam der geliefde voor een vertrouwd oor te mogen uitspreken. En toch moest hij laaghartig en zelfzuchtig schijnen, en toch gevoelde hij dat de ander niet meer gelooven kon aan de goedheid van zijn hart, toen hij antwoordde, zooals zijn gegeven woord het eischte:

‘Neen, Arundel, neen! ik kan haar niet helpen; voor haar vermag ik niets; vooralsnog niets;... misschien in het vervolg;... wie weet.... geloof mij, mijn vriend, laat ons geduld hebben.’

‘Dus heeft de beklagenswaardige ook u verloren,’ hervatte de Groot-Kamerheer koel en met stugheid. ‘Ik moet u nog met eene vraag lastig vallen, Mylord; het is de laatste over dit onderwerp. Zult gij mij tegen zijn, zoo ik in haar voordeel spreek?’

‘Op mijn ridderwoord, neen!’ riep Devonshire schielijk; zijne oogen schitterden van vreugde over eene uitkomst, die zoozeer met zijne wenschen instemde en hij stond op en vatte Arundel bij de hand. Zoo brandend jaagde het bloed hem doo

In [30]:
for a in get_annotations_of_type('page', all_annotations):
    print(a)

{'label': 'page', 'begin_anchor': 'anchor_cc1ad4bf-1f43-4939-9b51-d997ae160791', 'end_anchor': 'anchor_0e0e0f2e-dea3-49a9-8802-f5bd17b41d54', 'id': ''}
{'label': 'page', 'begin_anchor': 'anchor_0e0e0f2e-dea3-49a9-8802-f5bd17b41d54', 'end_anchor': 'anchor_be024c32-5df7-4266-9678-f4682bae0a69', 'id': 'page-IV'}
{'label': 'page', 'begin_anchor': 'anchor_be024c32-5df7-4266-9678-f4682bae0a69', 'end_anchor': 'anchor_be024c32-5df7-4266-9678-f4682bae0a69', 'id': 'page-1'}
{'label': 'page', 'begin_anchor': 'anchor_be024c32-5df7-4266-9678-f4682bae0a69', 'end_anchor': 'anchor_48561225-98d6-4e71-8313-8105840bb125', 'id': 'page-3'}
{'label': 'page', 'begin_anchor': 'anchor_48561225-98d6-4e71-8313-8105840bb125', 'end_anchor': 'anchor_6b055bda-c16a-4e25-95ae-561a3cef6c1a', 'id': 'page-4'}
{'label': 'page', 'begin_anchor': 'anchor_6b055bda-c16a-4e25-95ae-561a3cef6c1a', 'end_anchor': 'anchor_626b7bab-697f-4488-9396-14586edbe5ea', 'id': 'page-5'}
{'label': 'page', 'begin_anchor': 'anchor_626b7bab-697f-4

In [19]:
import json

# write all_textlines to a file
with open('all_tei_textlines.txt', 'w') as filehandle:
    json.dump(all_textelements, filehandle)

In [20]:
for a in get_annotations_of_type('page', all_annotations):
    print(json.dumps(a, sort_keys=False, indent=2))

{
  "label": "page",
  "begin_index": 0,
  "end_index": 295,
  "id": ""
}
{
  "label": "page",
  "begin_index": 295,
  "end_index": 297,
  "id": "page-IV"
}
{
  "label": "page",
  "begin_index": 297,
  "end_index": 297,
  "id": "page-1"
}
{
  "label": "page",
  "begin_index": 297,
  "end_index": 298,
  "id": "page-3"
}
{
  "label": "page",
  "begin_index": 298,
  "end_index": 302,
  "id": "page-4"
}
{
  "label": "page",
  "begin_index": 302,
  "end_index": 315,
  "id": "page-5"
}
{
  "label": "page",
  "begin_index": 315,
  "end_index": 319,
  "id": "page-6"
}
{
  "label": "page",
  "begin_index": 319,
  "end_index": 325,
  "id": "page-7"
}
{
  "label": "page",
  "begin_index": 325,
  "end_index": 331,
  "id": "page-8"
}
{
  "label": "page",
  "begin_index": 331,
  "end_index": 334,
  "id": "page-9"
}
{
  "label": "page",
  "begin_index": 334,
  "end_index": 344,
  "id": "page-10"
}
{
  "label": "page",
  "begin_index": 344,
  "end_index": 350,
  "id": "page-11"
}
{
  "label": "page",


In [21]:
for a in get_annotations_of_type('paragraph', all_annotations):
    print(json.dumps(a, sort_keys=False, indent=2))

{
  "label": "paragraph",
  "begin_index": 0,
  "end_index": 0,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 1,
  "end_index": 1,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2,
  "end_index": 2,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 3,
  "end_index": 3,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 4,
  "end_index": 14,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 15,
  "end_index": 15,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 16,
  "end_index": 16,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 17,
  "end_index": 17,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 18,
  "end_index": 18,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 19,
  "end_index": 21,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 22,
  "end_index": 22,
  "id": "gene

}
{
  "label": "paragraph",
  "begin_index": 2420,
  "end_index": 2420,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2421,
  "end_index": 2421,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2422,
  "end_index": 2422,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2423,
  "end_index": 2423,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2424,
  "end_index": 2424,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2425,
  "end_index": 2425,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2426,
  "end_index": 2426,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2427,
  "end_index": 2427,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2428,
  "end_index": 2428,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2429,
  "end_index": 2431,
  "id": "generate_later"
}
{
  "label": "paragraph",
  